## 목차
### 1. Library Import
### 2. 학습데이터 생성
### 3. Light-gbm 모델 훈련
### 4. 교차 검증 점수 확인
### 5. 제출 파일 생성

# 1. Library Import

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')
from google.colab import drive
drive.mount('/content/gdriv')
PATH ='/content/gdriv/My Drive/LG/data/'
# 필요한 함수 정의
def make_datetime(x):
    # string 타입의 Time column을 datetime 타입으로 변경
    x     = str(x)
    year  = int(x[:4])
    month = int(x[4:6])
    day   = int(x[6:8])
    hour  = int(x[8:10])
    #mim  = int(x[10:12])
    #sec  = int(x[12:])
    return dt.datetime(year, month, day, hour)

def string2num(x):
    # (,)( )과 같은 불필요한 데이터 정제
    x = re.sub(r"[^0-9]+", '', str(x))
    if x =='':
        return 0
    else:
        return int(x)




Mounted at /content/gdriv


# 2. 학습 데이터 생성

## 2.1 train_err

* train_err_data.csv 파일의 errtype column의 value_counts를 카운트하여 학습에 사용.  
* 주어진 기간동안 user_id별로 peroblem가 한 번이라도 발생했는 지를 확인.  

In [ ]:
train_err  = pd.read_csv(PATH+'train_err_data.csv')
display(train_err.head())

user_id            time model_nm       fwver  errtype errcode
0    10000  20201101025616  model_3  05.15.2138       15       1
1    10000  20201101030309  model_3  05.15.2138       12       1
2    10000  20201101030309  model_3  05.15.2138       11       1
3    10000  20201101050514  model_3  05.15.2138       16       1
4    10000  20201101050515  model_3  05.15.2138        4       0

In [ ]:
# 데이터 설명을 확인하면
# ueser_id가 10000부터 24999까지 총 15000개가 연속적으로 존재.
display(train_err.head())
train_user_id_max = 24999
train_user_id_min = 10000
train_user_number = 15000

user_id            time model_nm       fwver  errtype errcode
0    10000  20201101025616  model_3  05.15.2138       15       1
1    10000  20201101030309  model_3  05.15.2138       12       1
2    10000  20201101030309  model_3  05.15.2138       11       1
3    10000  20201101050514  model_3  05.15.2138       16       1
4    10000  20201101050515  model_3  05.15.2138        4       0

In [ ]:
print(np.sort(train_err.errtype.unique()))
# errtype이 1부터 42까지 29를 제외한 41개가 존재.

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 30 31 32 33 34 35 36 37 38 39 40 41 42]


In [ ]:
# errtype_pick = [16, 26, 31, 40, 15, 5, 23, 14, 22, 34, 4, 12, 17, 42, 11, 33]

In [ ]:
# train_err['errtype_pick'] = train_err['errtype'].isin(errtype_pick).astype('int64')

In [ ]:
# train_err = train_err[train_err.errtype_pick == 1]
# train_err

In [ ]:
# user_id와 errtype만을 사용하여 데이터 셋 생성
# 모든 일자에 대해서 errtype별 발생 건수를 count
# pandas의 groupby를 활용할 경우 큰 연산 자원이 필요.
# numpy로 placeholder를 만들어 구현함.
id_error = train_err[['user_id','errtype']].values
error = np.zeros((train_user_number,42))

for person_idx, err in tqdm(id_error):
    # person_idx - train_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    error[person_idx - train_user_id_min,err - 1] += 1
error.shape

100%|██████████| 16554663/16554663 [00:47<00:00, 346941.79it/s]


(15000, 42)

In [ ]:
train_err.dropna(axis = 0, inplace=True)
train_err.shape

(16554662, 6)

In [ ]:
items = train_err.errcode.unique().astype(str)
items

array(['1', '0', '2', ..., '4526', '3965', '25999'], dtype='<U39')

In [ ]:
len(train_err.errcode.unique())

2805

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit(items)
labels = encoder.transform(train_err.errcode)
labels.shape

(16554662,)

In [ ]:
train_err['errcode_label'] = labels

In [ ]:
errcode_pick = [2, 4, 279,370,2566, 2728, 196, 2796, 2755, 
                2799, 2803, 2771, 2517, 1209, 2500, 176, 
                2798, 2518, 2804, 2768, 2791, 297, 2802, 
                2492, 2776, 2785, 2800, 2598, 2579, 2744, 
                2763, 2639, 2774, 2758, 2729, 1537, 2153, 2676]

In [ ]:
train_err['errcode_pick'] = train_err['errcode_label'].isin(errcode_pick).astype('int64')

In [ ]:
train_err.errcode_pick.value_counts()

1    16311122
0      243540
Name: errcode_pick, dtype: int64

In [ ]:
train_err = train_err[train_err.errcode_pick == 1]
train_err

user_id            time model_nm  ... errcode  errcode_label errcode_pick
0           10000  20201101025616  model_3  ...       1              4            1
1           10000  20201101030309  model_3  ...       1              4            1
2           10000  20201101030309  model_3  ...       1              4            1
3           10000  20201101050514  model_3  ...       1              4            1
4           10000  20201101050515  model_3  ...       0              2            1
...           ...             ...      ...  ...     ...            ...          ...
16554658    24999  20201130163051  model_3  ...       1              4            1
16554659    24999  20201130172625  model_3  ...       1              4            1
16554660    24999  20201130172625  model_3  ...       0              2            1
16554661    24999  20201130172631  model_3  ...       0              2            1
16554662    24999  20201130210625  model_3  ...       1              4            1

[16311122 rows x 8 columns]

In [ ]:
train_err.errcode_label.unique()

array([   4,    2,  279, 2755, 2728,  196, 1209,  176,  370, 2517, 2803,
       2796, 2768, 2799, 2771, 2804, 2798, 2518, 2500, 2598, 2579, 2800,
       2492, 2785, 2639, 2791, 2774, 2566, 2676, 2776, 2153, 1537, 2729,
        297, 2744, 2802, 2758, 2763])

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder2 = LabelEncoder()
encoder2.fit(train_err.errcode_label.unique())
labels = encoder2.transform(train_err.errcode_label)
labels.shape

(16311122,)

In [ ]:
train_err['errcode_label_2'] = labels

In [ ]:
train_err.errcode_label_2.value_counts().sort_index()

0     2599123
1     8906967
2       12040
3      263577
4      166382
5         982
6       91223
7       20249
8        7335
9        7392
10       5028
11     332422
12       9317
13     334018
14       2087
15     129876
16       4023
17      65992
18      23452
19     897863
20        172
21        163
22     104138
23        105
24        157
25      11959
26      34631
27        138
28       1583
29       3914
30       1740
31     219238
32     100940
33    1835600
34       4011
35        392
36     110370
37       2523
Name: errcode_label_2, dtype: int64

In [ ]:
len(train_err.errcode_label_2.unique())

38

In [ ]:
id_code = train_err[['user_id','errcode_label_2']].values
code = np.zeros((train_user_number,38))

for person_idx, co in tqdm(id_code):
    # person_idx - train_user_id_min 위치에 person_idx, errcode에 해당하는 error값을 +1
    code[person_idx - train_user_id_min,co] += 1
code.shape

100%|██████████| 16311122/16311122 [00:41<00:00, 391237.74it/s]


(15000, 38)

In [ ]:
type_code = np.append(error, code, axis=1)
type_code.shape

(15000, 80)

In [ ]:
encoder_fw = LabelEncoder()
encoder_fw.fit(train_err.fwver.unique())
labels = encoder_fw.transform(train_err.fwver)
labels.shape

(16311122,)

In [ ]:
train_err['fwver_label'] = labels

In [ ]:
train_err.fwver_label.value_counts().sort_index()

0          29
1        2113
2     1505606
3         296
4         101
5         140
6     5012473
7         869
8      137635
9        2522
10         39
11          4
12       5423
13    2871983
14    1293000
15        909
16        408
17       3236
18         58
19     962201
20    2500628
21      33533
22       2459
23      23404
24         78
25       8592
26         12
27         18
28          9
29        705
30         23
31    1905811
32        191
33      31831
34       3044
35       1528
36        211
Name: fwver_label, dtype: int64

In [ ]:
len(train_err.fwver_label.unique())

37

In [ ]:
id_fwver = train_err[['user_id','fwver_label']].values
fwver = np.zeros((train_user_number,37))

for person_idx, fw in tqdm(id_fwver):
    # person_idx - train_user_id_min 위치에 person_idx, errcode에 해당하는 error값을 +1
    fwver[person_idx - train_user_id_min,fw] += 1
fwver.shape

100%|██████████| 16311122/16311122 [00:41<00:00, 395464.82it/s]


(15000, 37)

In [ ]:
type_code_fwver = np.append(type_code, fwver, axis=1)
type_code_fwver.shape

(15000, 117)

In [ ]:
#참고) 일별 합산 데이터 생성 코드      
'''
train_err             = pd.read_csv(PATH+'train_err_data.csv')
train_err['datetime'] = train_err['time'].apply(make_datetime)

min_day = train_err.datetime.min().date()

train_err['days'] = train_err.loc[:,'datetime'].dt.date - min_day
train_err['days'] = train_err['days'].dt.days
display(train_err)

id_error = train_err[['user_id','errtype','days']].values
error = np.zeros((train_user_number,42, 33))
for idx, err, days in tqdm(id_error):
    error[idx - train_user_id_min,err - 1, days] += 1
error.shape
'''

"\ntrain_err             = pd.read_csv(PATH+'train_err_data.csv')\ntrain_err['datetime'] = train_err['time'].apply(make_datetime)\n\nmin_day = train_err.datetime.min().date()\n\ntrain_err['days'] = train_err.loc[:,'datetime'].dt.date - min_day\ntrain_err['days'] = train_err['days'].dt.days\ndisplay(train_err)\n\nid_error = train_err[['user_id','errtype','days']].values\nerror = np.zeros((train_user_number,42, 33))\nfor idx, err, days in tqdm(id_error):\n    error[idx - train_user_id_min,err - 1, days] += 1\nerror.shape\n"

## 2.2 problem

In [ ]:
train_prob = pd.read_csv(PATH+'train_problem_data.csv')
problem = np.zeros(15000)
# error와 동일한 방법으로 person_idx - 10000 위치에 
# person_idx의 problem이 한 번이라도 발생했다면 1
# 없다면 0
problem[train_prob.user_id.unique()-10000] = 1 
problem.shape

(15000,)

# 3. Light-gbm 모델 훈련

In [1]:
# 변수 이름 변경
# error  -> train_x
# problem-> train_y

train_x = type_code_fwver
train_y = problem
del type_code_fwver, problem, code, error
print(train_x.shape)
print(train_y.shape)

NameError: ignored

In [2]:
# Train
#-------------------------------------------------------------------------------------
# validation auc score를 확인하기 위해 정의
def f_pr_auc(probas_pred, y_true):
    labels=y_true.get_label()
    p, r, _ = precision_recall_curve(labels, probas_pred)
    score=auc(r,p) 
    return "pr_auc", score, True
#-------------------------------------------------------------------------------------
models     = []
recalls    = []
precisions = []
auc_scores   = []
threshold = 0.5
# 파라미터 설정
params =      {
                'boosting_type' : 'gbdt',
                'objective'     : 'binary',
                'metric'        : 'auc',
                'seed': 1015
                }
#-------------------------------------------------------------------------------------
# 5 Kfold cross validation
k_fold = KFold(n_splits=5, shuffle=True, random_state=42)
for train_idx, val_idx in k_fold.split(train_x):

    # split train, validation set
    X = train_x[train_idx]
    y = train_y[train_idx]
    valid_x = train_x[val_idx]
    valid_y = train_y[val_idx]

    d_train= lgb.Dataset(X, y)
    d_val  = lgb.Dataset(valid_x, valid_y)
    
    #run traning
    model = lgb.train(
                        params,
                        train_set       = d_train,
                        num_boost_round = 2000,
                        valid_sets      = d_val,
                        feval           = f_pr_auc,
                        verbose_eval    = 20, 
                        early_stopping_rounds = 100
                       )
    
    # cal valid prediction
    valid_prob = model.predict(valid_x)
    valid_pred = np.where(valid_prob > threshold, 1, 0)
    
    # cal scores
    recall    = recall_score(    valid_y, valid_pred)
    precision = precision_score( valid_y, valid_pred)
    auc_score = roc_auc_score(   valid_y, valid_prob)

    # append scores
    models.append(model)
    recalls.append(recall)
    precisions.append(precision)
    auc_scores.append(auc_score)

    print('==========================================================')

NameError: ignored

# 4. 교차검증 점수 확인

In [ ]:
print(np.mean(auc_scores))

0.8192664291641979


# 5. 제출 파일 생성

In [ ]:
test_err  = pd.read_csv(PATH+'test_err_data.csv')
display(test_err.head())

user_id            time model_nm       fwver  errtype errcode
0    30000  20201101030227  model_1  04.16.3553       31       1
1    30000  20201101030227  model_1  04.16.3553       33       2
2    30000  20201101030228  model_1  04.16.3553       15       1
3    30000  20201101030256  model_1  04.16.3553       22       1
4    30000  20201101030300  model_1  04.16.3553       11       1

In [ ]:
# 데이터 설명을 확인하면
# test 데이터는 ueser_id가 30000부터 44998까지 총 14999개가 존재.
test_user_id_max = 44998
test_user_id_min = 30000
test_user_number = 14999

In [ ]:
test_err.dropna(axis = 0, inplace=True)
test_err.shape

(16532644, 6)

In [ ]:
id_error = test_err[['user_id','errtype']].values
test_x = np.zeros((test_user_number,42))
for person_idx, err in tqdm(id_error):
    # person_idx - test_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    test_x[person_idx - test_user_id_min,err - 1] += 1
test_x = test_x.reshape(test_x.shape[0],-1)
print(test_x.shape)

100%|██████████| 16532644/16532644 [00:47<00:00, 350756.72it/s]

(14999, 42)


In [ ]:
test_err

user_id            time model_nm       fwver  errtype errcode
0           30000  20201101030227  model_1  04.16.3553       31       1
1           30000  20201101030227  model_1  04.16.3553       33       2
2           30000  20201101030228  model_1  04.16.3553       15       1
3           30000  20201101030256  model_1  04.16.3553       22       1
4           30000  20201101030300  model_1  04.16.3553       11       1
...           ...             ...      ...         ...      ...     ...
16532643    44998  20201130210050  model_1  04.16.3553       40       0
16532644    44998  20201130211831  model_1  04.16.3553       31       1
16532645    44998  20201130211832  model_1  04.16.3553       15       1
16532646    44998  20201130212259  model_1  04.16.3553       16       1
16532647    44998  20201130212316  model_1  04.16.3553       31       0

[16532644 rows x 6 columns]

In [ ]:
a = set(test_err.errcode).intersection(train_err.errcode)
print(a)

{'80', 'connection timeout', '6', 'NFANDROID2', 'H-51046', '79', 'S-64002', 'B-A8002', 'scanning timeout', '1', 'connectionterminated by local host', 'Q-64002', 'UNKNOWN', '5', '83', '4', '95', '3', '86', '21', 'C-11017', '2', 'standby', 'P-41007 ', 'S-65002', 'connection fail to establish', '14', 'active', 'S-61001', 'V-21008', '8.0', 'PHONE_ERR', '78', '90', 'terminate by peer user', '0', '13', '84'}


In [ ]:
a = list(a)
print(a)

['80', 'connection timeout', '6', 'NFANDROID2', 'H-51046', '79', 'S-64002', 'B-A8002', 'scanning timeout', '1', 'connectionterminated by local host', 'Q-64002', 'UNKNOWN', '5', '83', '4', '95', '3', '86', '21', 'C-11017', '2', 'standby', 'P-41007 ', 'S-65002', 'connection fail to establish', '14', 'active', 'S-61001', 'V-21008', '8.0', 'PHONE_ERR', '78', '90', 'terminate by peer user', '0', '13', '84']


In [ ]:
test_err.errcode.unique()

array(['1', '2', 'connection timeout', ..., '5803', '5444', '5691'],
      dtype=object)

In [ ]:
test_err['errcode_pick'] = test_err['errcode'].isin(a).astype('int64')

In [ ]:
test_err.errcode_pick.value_counts()

1    16292981
0      239663
Name: errcode_pick, dtype: int64

In [ ]:
test_err = test_err[test_err.errcode_pick == 1]
test_err

user_id            time model_nm  ... errtype  errcode errcode_pick
0           30000  20201101030227  model_1  ...      31        1            1
1           30000  20201101030227  model_1  ...      33        2            1
2           30000  20201101030228  model_1  ...      15        1            1
3           30000  20201101030256  model_1  ...      22        1            1
4           30000  20201101030300  model_1  ...      11        1            1
...           ...             ...      ...  ...     ...      ...          ...
16532643    44998  20201130210050  model_1  ...      40        0            1
16532644    44998  20201130211831  model_1  ...      31        1            1
16532645    44998  20201130211832  model_1  ...      15        1            1
16532646    44998  20201130212259  model_1  ...      16        1            1
16532647    44998  20201130212316  model_1  ...      31        0            1

[16292981 rows x 7 columns]

In [ ]:
labels = encoder.transform(test_err.errcode)
labels.shape

(16292981,)

In [ ]:
test_err['errcode_label'] = labels

In [ ]:
errcode_pick = [2, 4, 279,370,2566, 2728, 196, 2796, 2755, 
                2799, 2803, 2771, 2517, 1209, 2500, 176, 
                2798, 2518, 2804, 2768, 2791, 297, 2802, 
                2492, 2776, 2785, 2800, 2598, 2579, 2744, 
                2763, 2639, 2774, 2758, 2729, 1537, 2153, 2676]

In [ ]:
test_err['errcode_pick'] = test_err['errcode_label'].isin(errcode_pick).astype('int64')

In [ ]:
test_err = test_err[test_err.errcode_pick == 1]
test_err

user_id            time model_nm  ... errcode  errcode_pick errcode_label
0           30000  20201101030227  model_1  ...       1             1             4
1           30000  20201101030227  model_1  ...       2             1           279
2           30000  20201101030228  model_1  ...       1             1             4
3           30000  20201101030256  model_1  ...       1             1             4
4           30000  20201101030300  model_1  ...       1             1             4
...           ...             ...      ...  ...     ...           ...           ...
16532643    44998  20201130210050  model_1  ...       0             1             2
16532644    44998  20201130211831  model_1  ...       1             1             4
16532645    44998  20201130211832  model_1  ...       1             1             4
16532646    44998  20201130212259  model_1  ...       1             1             4
16532647    44998  20201130212316  model_1  ...       0             1             2

[16292981 rows x 8 columns]

In [ ]:
test_err.errcode_label.value_counts().sort_index()

2       2565532
4       8750154
176       11740
196      316110
279      155401
297         997
370       86451
1209      19084
1537      12991
2153      13030
2492       5059
2500     324186
2517       9427
2518     326179
2566       1836
2579     140642
2598       3164
2639      58820
2676      17806
2728     855747
2729        211
2744        113
2755     108244
2758        142
2763        181
2768      11808
2771      35026
2774        136
2776       1636
2785       3571
2791       1064
2796     225519
2798     109690
2799    1890632
2800      84809
2802        347
2803     139947
2804       5549
Name: errcode_label, dtype: int64

In [ ]:
labels = encoder2.transform(test_err.errcode_label)
labels.shape

(16292981,)

In [ ]:
test_err['errcode_label_2'] = labels

In [ ]:
test_err.errcode_label_2.value_counts().sort_index()

0     2565532
1     8750154
2       11740
3      316110
4      155401
5         997
6       86451
7       19084
8       12991
9       13030
10       5059
11     324186
12       9427
13     326179
14       1836
15     140642
16       3164
17      58820
18      17806
19     855747
20        211
21        113
22     108244
23        142
24        181
25      11808
26      35026
27        136
28       1636
29       3571
30       1064
31     225519
32     109690
33    1890632
34      84809
35        347
36     139947
37       5549
Name: errcode_label_2, dtype: int64

In [ ]:
id_code = test_err[['user_id','errcode_label_2']].values
test_code = np.zeros((test_user_number,38))

for person_idx, code in tqdm(id_code):
    # person_idx - test_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    test_code[person_idx - test_user_id_min, code] += 1
test_code = test_code.reshape(test_code.shape[0],-1)
print(test_code.shape)

100%|██████████| 16292981/16292981 [00:41<00:00, 390846.95it/s]

(14999, 38)


In [ ]:
test_type_code = np.append(test_x, test_code, axis=1)
test_type_code.shape

(14999, 80)

In [ ]:
a = set(test_err.fwver).intersection(train_err.fwver)
print(a)

{'10', '04.16.3571', '04.73.2571', '04.82.1684', '05.15.2138', '04.22.1778', '04.82.1778', '04.16.3439', '03.11.1167', '04.33.1125', '04.16.3569', '8.5.3', '05.15.3104', '05.15.2120', '05.15.2092', '03.11.1141', '04.82.1730', '05.15.2114', '05.66.3237', '04.33.1149', '04.22.1666', '03.11.1149', '04.33.1185', '04.33.1171', '04.22.1684', '05.66.3571', '04.22.1750', '04.22.1656', '04.33.1261', '04.16.3553', '04.73.2237'}


In [ ]:
a = list(a)
print(a)

['10', '04.16.3571', '04.73.2571', '04.82.1684', '05.15.2138', '04.22.1778', '04.82.1778', '04.16.3439', '03.11.1167', '04.33.1125', '04.16.3569', '8.5.3', '05.15.3104', '05.15.2120', '05.15.2092', '03.11.1141', '04.82.1730', '05.15.2114', '05.66.3237', '04.33.1149', '04.22.1666', '03.11.1149', '04.33.1185', '04.33.1171', '04.22.1684', '05.66.3571', '04.22.1750', '04.22.1656', '04.33.1261', '04.16.3553', '04.73.2237']


In [ ]:
test_err['fwver_pick'] = test_err['fwver'].isin(a).astype('int64')

In [ ]:
test_err = test_err[test_err.fwver_pick == 1]
test_err

user_id            time  ... errcode_label_2 fwver_pick
0           30000  20201101030227  ...               1          1
1           30000  20201101030227  ...               4          1
2           30000  20201101030228  ...               1          1
3           30000  20201101030256  ...               1          1
4           30000  20201101030300  ...               1          1
...           ...             ...  ...             ...        ...
16532643    44998  20201130210050  ...               0          1
16532644    44998  20201130211831  ...               1          1
16532645    44998  20201130211832  ...               1          1
16532646    44998  20201130212259  ...               1          1
16532647    44998  20201130212316  ...               0          1

[16286618 rows x 10 columns]

In [ ]:
labels = encoder_fw.transform(test_err.fwver)
labels.shape

(16286618,)

In [ ]:
test_err['fwver_label'] = labels

In [ ]:
test_err.fwver_label.value_counts().sort_index()

0        1569
1        4552
2     1209618
5         362
6     5106699
7         395
8      138948
10        835
11         43
12       4948
13    2966261
14    1367084
16         84
17       6018
18         40
19     986278
20    2603666
21      20409
22       4233
23      23002
24        268
25      12002
27          4
28        123
29        425
31    1799245
32        808
33      25683
34       2442
35        561
36         13
Name: fwver_label, dtype: int64

In [ ]:
id_fwver = test_err[['user_id','fwver_label']].values
test_fwver = np.zeros((test_user_number,37))

for person_idx, fwv in tqdm(id_fwver):
    # person_idx - test_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    test_fwver[person_idx - test_user_id_min, fwv] += 1
test_fwver = test_fwver.reshape(test_fwver.shape[0],-1)
print(test_fwver.shape)

100%|██████████| 16286618/16286618 [00:41<00:00, 393441.23it/s]

(14999, 37)


In [ ]:
test_type_code_fw = np.append(test_type_code, test_fwver, axis=1)
test_type_code_fw.shape

(14999, 117)

In [ ]:
# 예측
pred_y_list = []
for model in models:
    pred_y = model.predict(test_type_code_fw)
    pred_y_list.append(pred_y.reshape(-1,1))
    
pred_ensemble = np.mean(pred_y_list, axis = 0)

In [ ]:
pred_ensemble

array([[0.89143781],
       [0.23268294],
       [0.56776787],
       ...,
       [0.74971688],
       [0.86082378],
       [0.43092948]])

In [ ]:
sample_submssion = pd.read_csv(PATH+'sample_submission.csv')

In [ ]:
sample_submssion['problem'] = pred_ensemble.reshape(-1)

In [ ]:
sample_submssion.to_csv(PATH+"dacon_baseline_errcode_type_code_pick_fwver38.csv", index = False)
sample_submssion

user_id   problem
0        30000  0.891438
1        30001  0.232683
2        30002  0.567768
3        30003  0.778287
4        30004  0.845711
...        ...       ...
14994    44994  0.156559
14995    44995  0.312692
14996    44996  0.749717
14997    44997  0.860824
14998    44998  0.430929

[14999 rows x 2 columns]